# Data Manipulation and Wrangling

Let's read in the same elections data from the previous exercise and do some data manipulation and wrangling using pandas.

In [3]:
import pandas as pd 
url = "https://raw.githubusercontent.com/fahadsultan/csc272/main/data/elections.csv"
elections = pd.read_csv(url)

## Data Alignment

`pandas` can make it much simpler to work with objects that have different indexes. For example, when you add objects, if any index pairs are not the same, the respective index in the result will be the union of the index pairs. Let’s look at an example:


In [3]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=["a", "c", "d", "e"])

s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=["a", "c", "e", "f", "g"])

s1, s2, s1 + s2

(a    7.3
 c   -2.5
 d    3.4
 e    1.5
 dtype: float64,
 a   -2.1
 c    3.6
 e   -1.5
 f    4.0
 g    3.1
 dtype: float64,
 a    5.2
 c    1.1
 d    NaN
 e    0.0
 f    NaN
 g    NaN
 dtype: float64)

The internal data alignment introduces missing values in the label locations that don’t overlap. Missing values will then propagate in further arithmetic computations.

In the case of DataFrame, alignment is performed on both rows and columns:

In [ ]:
df1 = pd.DataFrame({"A": [1, 2], "B":[3, 4]})
df2 = pd.DataFrame({"B": [5, 6], "D":[7, 8]})
df1 + df2

## Math Operations

In native Python, we have a number of operators that we can use to manipulate data. Most, if not all, of these operators can be used with Pandas Series and DataFrames and are applied element-wise in parallel. A summary of the operators supported by Pandas is shown below:

| Category | Operators | Supported by Pandas | Comments |
| --- | --- | --- | --- |
| Arithmetic | `+`, `-`, `*`, `/`, `%`, `//`, `**` | ✅ | Assuming comparable shapes (equal length) |
| Assignment | `=`, `+=`, `-=`, `*=`, `/=`, `%=`, `//=`, `**=` | ✅ | Assuming comparable shapes |
| Comparison | `==`, `!=`, `>`, `<`, `>=`, `<=` | ✅ | Assuming comparable shapes |
| Logical | `and`, `or`, `not` | ❌ | Use `&`, `\|`, `~` instead |
| Identity | `is`, `is not` | ✅ | Assuming comparable data type/structure |
| Membership | `in`, `not in` | ❌ | Use `isin()` method instead |
| Bitwise | `&`, `\|`, `^`, `~`, `<<`, `>>` | ❌ |  |

The most significant difference is that <u><b>logical operators `and`, `or`, and `not` are NOT used with Pandas Series and DataFrames</u></b>. Instead, we use `&`, `|`, and `~` respectively.

Membership operators `in` and `not in` are also not used with Pandas Series and DataFrames. Instead, we use the `isin()` method.


## Creating new columns

Creating new columns in a DataFrame is a common task when working with data. In this notebook, we will see how to create new columns in a DataFrame based on existing columns or other values.

<center><img src="https://pandas.pydata.org/docs/_images/05_newcolumn_1.svg" width="80%" style="filter:invert(1)" /></center>

New columns can be created by assigning a value to a new column name. For example, to create a new column named `new_column` with a constant value `10`, we can use the following code:


In [13]:
elections['constant'] = 10

elections.head()

,Year,Candidate,Party,Popular vote,Result,%,constant
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,10
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,10
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,10
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,10
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,10


<br/>

If we want to create a new column based on an existing column, we can refer to the existing column by its name, within the square brackets, on the right side of the assignment operator. For example, to create a new column named `new_column` with the values of the existing column `column1`, we can use the following code:

<br/>


In [14]:
elections['total_voters'] = ((elections['Popular vote']* 100) / elections['%']).astype(int)

elections.head()

,Year,Candidate,Party,Popular vote,Result,%,constant,total_voters
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,10,264413
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,10,264412
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,10,1143702
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,10,1143703
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,10,1287655


Note that the new column will have the same length as the DataFrame and the calculations are element-wise. That is, the value of the new column at row `i` will be calculated based on the value of the existing column at row `i`.

<center><img src="https://pandas.pydata.org/docs/_images/05_newcolumn_2.svg" width="80%" style="filter:invert(1)" /></center>



These element-wise operations are vectorized and are very efficient.

## `.apply(f, axis=0/1)`

A frequent operation in `pandas` is applying a function on to either each column or row of a DataFrame. 

DataFrame’s `apply` method does exactly this. 


<center><img src="../assets/vectorized2.png" width="30%" style="filter:invert(1)" /></center>


Let's say we wanted to count the number of unique values that each column takes on. We can use `.apply` to answer that question: 

In [ ]:
def count_unique(col):
    return len(set(col))

elections.apply(count_unique, axis="index") # function is passed an individual column

Year             50
Candidate       132
Party            36
Popular vote    182
Result            2
%               182
dtype: int64

### Column-wise: `axis=0` (default)

`data.apply(f, axis=0)` applies the function `f` to <b><u>each column</u></b> of the DataFrame `data`. 

<center><img src="../assets/axis0b.png" width="100%" style="filter:invert(1)" /></center>

For example, if we wanted to find the number of unique values in each column of a DataFrame `data`, we could use the following code:


In [33]:
def count_unique(column):
    return len(column.unique())

elections.apply(count_unique, axis=0)

Year             50
Candidate       132
Party            36
Popular vote    182
Result            2
%               182
dtype: int64

### Row-wise: `axis=1`

`data.apply(f, axis=1)` applies the function `f` to <b><u>each row</u></b> of the DataFrame `data`.

<center><img src="../assets/axis1b.png" width="100%" style="filter:invert(1)" /></center>

For instance, let's say we wanted to count the total number of voters in an election. 

We can use `.apply` to answer that question using the following formula: 

$$ \text{total} \times \frac{\%}{100} = \text{Popular vote} $$

In [36]:
def compute_total(row):
    return int(row['Popular vote']*100/row['%'])

elections.apply(compute_total, axis=1)

0         264413
1         264412
2        1143702
3        1143703
4        1287655
         ...    
177    135720167
178    158383403
179    158383403
180    158383401
181    158383402
Length: 182, dtype: int64

## Summary Statistics

In data science, we often want to compute summary statistics. `pandas` provides a number of built-in methods for this purpose.

<center><img src="https://pandas.pydata.org/docs/_images/06_aggregate.svg" width="80%" style="filter:invert(1)" /></center>

For example, we can use the `.mean()`, `.median()` and `.std()` methods to compute the mean, median, and standard deviation of a column, respectively.

In [37]:
elections['%'].mean(), elections['%'].median(), elections['%'].std()

(27.470350372043967, 37.67789306, 22.96803399144419)

Similarly, we can use the `.max()` and `.min()` methods to compute the maximum and minimum values of a `Series` or `DataFrame`.

In [28]:
elections['%'].max(), elections['%'].min()

(61.34470329, 0.098088334)

The `.sum()` method computes the sum of all the values in a `Series` or `DataFrame`.

<center><img src="https://pandas.pydata.org/docs/_images/06_reduction.svg" width="80%" style="filter:invert(1)" /></center>


The `.describe()` method computes summary statistics for a `Series` or `DataFrame`. It computes the mean, standard deviation, minimum, maximum, and the quantiles of the data.

In [30]:
elections['%'].describe()

count    182.000000
mean      27.470350
std       22.968034
min        0.098088
25%        1.219996
50%       37.677893
75%       48.354977
max       61.344703
Name: %, dtype: float64

In [29]:
elections.describe()

,Year,Popular vote,%
count,182.000000,1.820000e+02,182.000000
mean,1934.087912,1.235364e+07,27.470350
std,57.048908,1.907715e+07,22.968034
min,1824.000000,1.007150e+05,0.098088
25%,1889.000000,3.876395e+05,1.219996
50%,1936.000000,1.709375e+06,37.677893
75%,1988.000000,1.897775e+07,48.354977
max,2020.000000,8.126892e+07,61.344703


## Other Handy Utility Functions

`pandas` contains an extensive library of functions that can help shorten the process of setting and getting information from its data structures. In the following section, we will give overviews of each of the main utility functions that will help us in in this course.

Discussing all functionality offered by `pandas` could take an entire semester! We will walk you through the most commonly-used functions, and encourage you to explore and experiment on your own. 

- `.shape`
- `.size`
- `.dtypes`
- `.astype()`
- `.describe() `
- `.sample()`
- `.value_counts()`
- `.unique()`
- `.sort_values()`

The `pandas` [documentation](https://pandas.pydata.org/docs/reference/index.html) will be a valuable resource.


### `.astype()`

Cast a pandas object to a specified dtype

In [ ]:
elections['%'].astype(int)

0      57
1      42
2      56
3      43
4      54
       ..
177     1
178    51
179    46
180     1
181     0
Name: %, Length: 182, dtype: int64

### `.describe()`

If many statistics are required from a `DataFrame` (minimum value, maximum value, mean value, etc.), then [`.describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) can be used to compute all of them at once. 


In [ ]:
elections.describe()

,Year,Popular vote,%
count,182.000000,1.820000e+02,182.000000
mean,1934.087912,1.235364e+07,27.470350
std,57.048908,1.907715e+07,22.968034
min,1824.000000,1.007150e+05,0.098088
25%,1889.000000,3.876395e+05,1.219996
50%,1936.000000,1.709375e+06,37.677893
75%,1988.000000,1.897775e+07,48.354977
max,2020.000000,8.126892e+07,61.344703


A different set of statistics will be reported if `.describe()` is called on a Series.

In [ ]:
elections["Party"].describe()

count            182
unique            36
top       Democratic
freq              47
Name: Party, dtype: object

In [ ]:
elections["Popular vote"].describe().astype(int)

count         182
mean     12353635
std      19077149
min        100715
25%        387639
50%       1709375
75%      18977751
max      81268924
Name: Popular vote, dtype: int64

### `.sample()`

As we will see later in the semester, random processes are at the heart of many data science techniques (for example, train-test splits, bootstrapping, and cross-validation). [`.sample()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) lets us quickly select random entries (a row if called from a DataFrame, or a value if called from a Series).

By default, `.sample()` selects entries *without* replacement. Pass in the argument `replace=True` to sample with replacement.

In [ ]:
# Sample a single row
elections.sample()

,Year,Candidate,Party,Popular vote,Result,%
135,1988,George H. W. Bush,Republican,48886597,win,53.518845


In [ ]:
# Sample 5 random rows
elections.sample(5)

,Year,Candidate,Party,Popular vote,Result,%
155,2000,Ralph Nader,Green,2882955,loss,2.741176
134,1984,Walter Mondale,Democratic,37577352,loss,40.729429
39,1884,Grover Cleveland,Democratic,4914482,win,48.884933
84,1928,Herbert Hoover,Republican,21427123,win,58.368524
177,2016,Jill Stein,Green,1457226,loss,1.073699


In [ ]:
# Randomly sample 4 names from the year 2000, with replacement
elections[elections["Result"] == "win"].sample(4, replace = True)

,Year,Candidate,Party,Popular vote,Result,%
53,1896,William McKinley,Republican,7112138,win,51.213817
131,1980,Ronald Reagan,Republican,43903230,win,50.897944
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
168,2012,Barack Obama,Democratic,65915795,win,51.258484


### `.value_counts()`

The [`Series.value_counts()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) methods counts the number of occurrence of each unique value in a `Series`. In other words, it *counts* the number of times each unique *value* appears. This is often useful for determining the most or least common entries in a `Series`.

In the example below, we can determine the name with the most years in which at least one person has taken that name by counting the number of times each name appears in the `"Name"` column of `elections`.

In [ ]:
elections["Party"].value_counts().head()

Democratic     47
Republican     41
Libertarian    12
Prohibition    11
Socialist      10
Name: Party, dtype: int64

### `.unique()`

If we have a Series with many repeated values, then [`.unique()`](https://pandas.pydata.org/docs/reference/api/pandas.unique.html) can be used to identify only the *unique* values. Here we return an array of all the names in `elections`. 

In [ ]:
elections["Result"].unique()

array(['loss', 'win'], dtype=object)

### `.drop_duplicates()`

If we have a DataFrame with many repeated rows, then [`.drop_duplicates()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html) can be used to remove the repeated rows.

Where `.unique()` only works with individual columns (Series) and returns an array of unique values, `.drop_duplicates()` can be used with multiple columns (DataFrame) and returns a DataFrame with the repeated rows removed.

In [9]:
elections[['Candidate', 'Party']].drop_duplicates()

,Candidate,Party
0,Andrew Jackson,Democratic-Republican
1,John Quincy Adams,Democratic-Republican
2,Andrew Jackson,Democratic
3,John Quincy Adams,National Republican
5,Henry Clay,National Republican
...,...,...
174,Evan McMullin,Independent
176,Hillary Clinton,Democratic
178,Joseph Biden,Democratic
180,Jo Jorgensen,Libertarian


### `.sort_values()`

Ordering a `DataFrame` can be useful for isolating extreme values. For example, the first 5 entries of a row sorted in descending order (that is, from highest to lowest) are the largest 5 values. [`.sort_values`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) allows us to order a `DataFrame` or `Series` by a specified column. We can choose to either receive the rows in `ascending` order (default) or `descending` order.

In [ ]:
# Sort the "Count" column from highest to lowest
elections.sort_values(by = "%", ascending=False).head()

,Year,Candidate,Party,Popular vote,Result,%
114,1964,Lyndon Johnson,Democratic,43127041,win,61.344703
91,1936,Franklin Roosevelt,Democratic,27752648,win,60.978107
120,1972,Richard Nixon,Republican,47168710,win,60.907806
79,1920,Warren Harding,Republican,16144093,win,60.574501
133,1984,Ronald Reagan,Republican,54455472,win,59.023326


We do not need to explicitly specify the column used for sorting when calling `.value_counts()` on a `Series`. We can still specify the ordering paradigm – that is, whether values are sorted in ascending or descending order.

In [ ]:
# Sort the "Name" Series alphabetically
elections["Candidate"].sort_values(ascending=True).head()

75     Aaron S. Watkins
27      Abraham Lincoln
23      Abraham Lincoln
108     Adlai Stevenson
105     Adlai Stevenson
Name: Candidate, dtype: object


### `.set_index()`

The [`.set_index()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) method is used to set the DataFrame index using existing columns.

In [40]:
elections.set_index("Candidate")

,Year,Party,Popular vote,Result,%
Candidate,,,,,
Andrew Jackson,1824,Democratic-Republican,151271,loss,57.210122
John Quincy Adams,1824,Democratic-Republican,113142,win,42.789878
Andrew Jackson,1828,Democratic,642806,win,56.203927
John Quincy Adams,1828,National Republican,500897,loss,43.796073
Andrew Jackson,1832,Democratic,702735,win,54.574789
...,...,...,...,...,...
Jill Stein,2016,Green,1457226,loss,1.073699
Joseph Biden,2020,Democratic,81268924,win,51.311515
Donald Trump,2020,Republican,74216154,loss,46.858542


### `.reset_index()`

The [`.reset_index()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) method is used to reset the index of a DataFrame. By default, the original index is stored in a new column called `index`.

In [41]:
elections.reset_index()

,index,Year,Candidate,Party,Popular vote,Result,%
0,0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...,...
177,177,2016,Jill Stein,Green,1457226,loss,1.073699
178,178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


### `.rename()`

The [`.rename()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) method is used to rename the columns or index labels of a DataFrame.

In [42]:
elections.rename(columns={"Candidate":"Name"})

,Year,Name,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979
